In [6]:
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import requests
import json
import re
import pandas as pd

In [7]:
from urllib import request
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException

In [8]:
from selenium.webdriver.common.keys import Keys

In [9]:
import numpy as np

In [13]:
idol_list = pd.read_csv('94to95_idol_list.csv')
idol_list.head()

,artist,artist_eng,member,leader,gender,agency,idol_id
0,룰라,Roo'Ra,4,이상민,혼,서울인 프로덕션,106610
1,쿨,Cool,3 (4),김성수,혼,쿨컴퍼니,100004
2,터보,Turbo,3 (2),김정남,남,더 터보 컴퍼니,100059
3,알이에프,R.ef,3,박철우,남,팀 엔터테인먼트,100015


In [14]:
idol_list.shape

(4, 7)

In [15]:
idol_list = idol_list[np.isfinite(idol_list['idol_id'])].reset_index(drop=True)

In [16]:
idol_list['idol_id'] = idol_list['idol_id'].astype(int)
idol_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 7 columns):
artist        4 non-null object
artist_eng    4 non-null object
member        4 non-null object
leader        4 non-null object
gender        4 non-null object
agency        4 non-null object
idol_id       4 non-null int32
dtypes: int32(1), object(6)
memory usage: 288.0+ bytes


In [17]:
# 수집을 위한 빈 리스트 생성

title = []
release_date = []
song_genre = []
lyrics = []
creator = []
artist = []
artist_m = []
debut_y = []
debut_t = []
agency = []

In [18]:
# 곡마다 가수 이름을 붙여주어야 하기에 가수 이름 리스트로 받아오기
names = idol_list.artist
names[0]

'룰라'

In [19]:
# 크롤러가 링크를 찾을 때 매번 바꿔주는 값. for문에 사용됩니다. 
ids = idol_list.idol_id
ids[0]

106610

In [20]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(r'C:\Users\Eunhoo\Code\NLP\chromedriver.exe')
driver.get("https://www.melon.com")

In [21]:
# 곡 정보를 찾아들어가기 위한 url을 분해해서 변수 지정해놓은 모습입니다. 
# l3 뒤에 붙는 숫자가 멜론 서버에 저장된 해당 가수의 해당 곡 번호입니다. 

l1 = 'https://www.melon.com/artist/song.htm?artistId='
# idol_id
l2 = '#params%5BlistType%5D=A&params%5BorderBy%5D=ISSUE_DATE&params%5BartistId%5D='
# idol_id
l3 = '&po=pageObj&startIndex='
# l1 + 'idol_id' + l2 + 'idol_id' + l3 + number

# 가수 이름을 넣기 위한 변수
i = 0
tmp = []
str_tmp = ""
arr = []

for id in ids:
    
    n = 1
    list_n = 1
    while True:
    #for k in range(1):
                 
        
        print('n: ',n)
        driver.get(l1+str(id)+l2+str(id)+l3+str(n))    
        sleep(2)
        
                
        # 곡 정보로 들어가기
        try:
            driver.find_element_by_xpath('//*[@id="conts"]/div[3]/ul/li[3]/a').click()
            str_tmp = '//*[@id="frm"]/div/table/tbody/tr[' + str(list_n) + ']/td[3]/div/div/a[1]'
            driver.find_element_by_xpath(str_tmp).click()
            print('list: ',list_n)
            # 발매날짜 크롤링
            try:
                str_tmp = driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[2]').text
                arr = str_tmp.split('.')
                print("check :",str_tmp)
                if( int(arr[0]) < 2000 and int(arr[0]) > 1993): # 96,97,98,99 발매 노래만
                    print("input: ", str_tmp)
                    release_date.append(str_tmp)
                    list_n += 1
                elif (int(arr[0]) < 1994): # 96년 이전 발매일 경우 break
                    break
                else:
                    list_n += 1
                    continue
                
            
            except NoSuchElementException as e:
                release_date.append('')
            
            # 제목 크롤링(제목은 없지 않기 때문에 예외 없음)
            title.append(driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[1]/div[1]').text)
            print('title:', title)

            try:
                song_genre.append(driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[3]').text)
            except NoSuchElementException as e:
                song_genre.append('')

            # 가사 크롤링
            try:
                lyrics.append(driver.find_element_by_xpath('//*[@id="d_video_summary"]').text)
            except NoSuchElementException as e:
                lyrics.append('')

            # 작사 작곡 내역이 없는 경우
            try:
                creator.append(driver.find_element_by_xpath('//*[@id="conts"]/div[3]/ul').text)
            except NoSuchElementException as e:
                creator.append('')

            n += 1
            
            # 가수 이름 붙이기
            artist.append(names[i])
            
            # 디버깅    
            print(len(title), title[len(title)-1])

        except NoSuchElementException as e:
            i += 1
            break

n:  1
list:  1
check : 2017.10.29
n:  1
list:  2
check : 2017.10.29
n:  1
list:  3
check : 2009.07.21
n:  1
list:  4
check : 2009.07.21
n:  1
list:  5
check : 2009.07.21
n:  1
list:  6
check : 2009.07.21
n:  1
list:  7
check : 2009.07.21
n:  1
list:  8
check : 2009.07.21
n:  1
list:  9
check : 2009.07.21
n:  1
list:  10
check : 2009.07.21
n:  1
list:  11
check : 2009.07.21
n:  1
list:  12
check : 2009.07.21
n:  1
list:  13
check : 2009.07.21
n:  1
list:  14
check : 2007.07.02
n:  1
list:  15
check : 2002.12.13
n:  1
list:  16
check : 2002.12.13
n:  1
list:  17
check : 2002.09.09
n:  1
list:  18
check : 2000.07.06
n:  1
list:  19
check : 2000.06.12
n:  1
list:  20
check : 2000.06.12
n:  1
list:  21
check : 2000.06.12
n:  1
list:  22
check : 2000.06.12
n:  1
list:  23
check : 2000.06.12
n:  1
list:  24
check : 2000.06.12
n:  1
list:  25
check : 2000.06.12
n:  1
list:  26
check : 2000.06.12
n:  1
list:  27
check : 2000.06.12
n:  1
list:  28
check : 2000.06.12
n:  1
list:  29
check : 2000.

list:  40
check : 2015.11.24
n:  1
list:  41
check : 2015.11.24
n:  1
list:  42
check : 2015.11.24
n:  1
list:  43
check : 2015.11.24
n:  1
list:  44
check : 2015.11.24
n:  1
list:  45
check : 2015.11.24
n:  1
list:  46
check : 2015.11.24
n:  1
list:  47
check : 2015.11.24
n:  1
list:  48
check : 2015.11.24
n:  1
list:  49
check : 2015.11.24
n:  1
list:  50
check : 2015.11.24
n:  1
n:  1
list:  1
check : 2013.09.25
n:  1
list:  2
check : 2013.09.10
n:  1
list:  3
check : 2013.09.10
n:  1
list:  4
check : 2013.09.10
n:  1
list:  5
check : 2013.07.23
n:  1
list:  6
check : 2012.09.25
n:  1
list:  7
check : 2012.09.25
n:  1
list:  8
check : 2011.12.29
n:  1
list:  9
check : 2011.12.19
n:  1
list:  10
check : 2005.09.06
n:  1
list:  11
check : 2004.07.28
n:  1
list:  12
check : 1999
input:  1999
title: ['6N6', '기도 (Good)', 'Moving', '고백', 'Spring (혼자한 사랑)', 'Late Night (밤이여)', 'Flora (향기로운 너)', '미워도 다시 한번 2', 'Adios', '변심', 'Take Me Home Country Road', '반추(反芻)', '100일째 만남', '비밀은 없어', '친구를 

In [22]:
test_list = [title, release_date, song_genre, creator, lyrics]
column_names = ['title', 'artist', 'release_date', 'song_genre', 'creator', 'lyrics']
column_v = [title, artist, release_date, song_genre, creator, lyrics]

test_df = pd.DataFrame(column_v)

In [23]:
test_p = test_df.head(6).T
test_p.head()

,0,1,2,3,4,5
0,6N6,룰라,1999.02,댄스,,
1,기도 (Good),룰라,1999.02,댄스,이상민\n작사\n최준영\n작사\n이상민\n작곡\n최준영\n작곡\n김준환\n편곡,Roo'Ra deeper lover\nlive lover live\n라라따리 라리라...
2,Moving,룰라,1999.02,댄스,백종열\n작사\n박근태\n작곡\n박근태\n편곡,가나봐 내가 떠나가도\n내가 가나봐\n그래봐라 네가 떠나가도\n아무렇지 않다고\n오...
3,고백,룰라,1999.02,댄스,이현도\n작사\n이현도\n작곡\n이현도\n편곡,내 사랑 그대여 이젠 말할게요\n때론 모진말로 아프게 하고\n때론 애태우게 만들었지...
4,Spring (혼자한 사랑),룰라,1999.02,댄스,홍재선\n작사\n홍재선\n작곡\n홍재선\n편곡,모두가 날 떠나도 내곁에 남아 있어요\n힘없이 나 그댈 보낼수 없어 울고 있어요\n...


In [24]:
test_p.columns = column_names
test_p.tail(350)

,title,artist,release_date,song_genre,creator,lyrics
0,6N6,룰라,1999.02,댄스,,
1,기도 (Good),룰라,1999.02,댄스,이상민\n작사\n최준영\n작사\n이상민\n작곡\n최준영\n작곡\n김준환\n편곡,Roo'Ra deeper lover\nlive lover live\n라라따리 라리라...
2,Moving,룰라,1999.02,댄스,백종열\n작사\n박근태\n작곡\n박근태\n편곡,가나봐 내가 떠나가도\n내가 가나봐\n그래봐라 네가 떠나가도\n아무렇지 않다고\n오...
3,고백,룰라,1999.02,댄스,이현도\n작사\n이현도\n작곡\n이현도\n편곡,내 사랑 그대여 이젠 말할게요\n때론 모진말로 아프게 하고\n때론 애태우게 만들었지...
4,Spring (혼자한 사랑),룰라,1999.02,댄스,홍재선\n작사\n홍재선\n작곡\n홍재선\n편곡,모두가 날 떠나도 내곁에 남아 있어요\n힘없이 나 그댈 보낼수 없어 울고 있어요\n...
5,Late Night (밤이여),룰라,1999.02,댄스,이상민\n작사\n이준\n작곡\n이준\n편곡,잊으려 애쓰는\n맘으로만 허무하기만\n이밤이 주위를 감싸 감싸 돌아\n눈에 눈물이 ...
6,Flora (향기로운 너),룰라,1999.02,댄스,김준환\n작사\n채리나\n작사\n김준환\n작곡\n김준환\n편곡,촉촉하게 스며드는 너\n난 너와 함께 하고 싶어\n나를 봐줘\n내 맘속엔 항상\n그...
7,미워도 다시 한번 2,룰라,1999.02,댄스,최준영\n작사\n이상민\n작사\n최준영\n작곡\n최준영\n편곡,언제나 같이 있자고\n영원히 함께 하자고\n눈물로써 맹세했던 네가\n성공해 날 떠났...
8,Adios,룰라,1999.02,발라드,이도연\n작사\n임기훈\n작곡\n임기훈\n편곡,그대여 날 봐요 그리고 웃어요 흘러내린 눈물에 추억들은\n지워지지 않게 이제야 알았...
9,변심,룰라,1999.02,댄스,이현규\n작사\n박근태\n작곡,정말 그럴줄 몰랐어 니가그럴줄 몰랐어\n그래도 난 너만을 믿었었는데\n언제는 내가 ...


In [25]:
test_p['title'].isnull().sum()

0

In [26]:
test_p.tail()

,title,artist,release_date,song_genre,creator,lyrics
33,그대만큼...,알이에프,1999,발라드,피정훈\n작사\n피정훈\n작곡\n피정훈\n편곡,이 세상에 그대만큼 아름다운\n노래가 있을 수 있다면\n나 매일 그대 앞에서 노래하...
34,Never Ending Story,알이에프,1999,댄스,이승호\n작사\n윤일상\n작곡\n윤일상\n편곡,여기저기 하루종일\n너를 찾아서 헤매어 봐도\n니 모습 간데 없고\n추억들만이 나를...
35,찬란한 사랑,알이에프,1999,댄스,홍재선\n작사\n홍재선\n작곡\n홍재선\n편곡,너에게 아무런 도움이\n되지 못했어\n내가 내 불행마저\n감당할 수 없는데\n그냥 ...
36,Closing Ment,알이에프,1999,댄스,,
37,부활을 기다리며,알이에프,1999,발라드,이승호\n작사\n윤일상\n작곡\n윤일상\n편곡,아닌거지 그렇지 말해봐\n다시 돌아오는 여행일 뿐이지\n내일이면 어제 그 미소로\n...


In [27]:
test_p.to_csv('k_pop_3.tsv',index=False,header=None,sep='\t')